In [2]:
import wandb
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torch

### Run other .ipybn files

In [ ]:
#%run NN_model.ipynb #later import as .py
#%run NN_dataset.ipynb #later import as .py

In [26]:
class CellDataset(Dataset):
    def __init__(self, data: pd.DataFrame, normalize:bool):
        # save predictros as DataFrame
        self.cell_descriptors = data.drop(columns=['target'])

        # Normalize target value to [-1, 1]
        res = data['target']
        if normalize:
            res = res.apply(lambda x: -1 if x == 0 else 1)

        # save target as DataFrame
        self.cell_response = res.astype('float64').to_frame()

    def __len__(self):
        return len(self.cell_descriptors)

    def __getitem__(self, idx):
        desc = self.cell_descriptors.iloc[idx]
        res = self.cell_response.iloc[idx]
        return desc.values, res.values
    
    def get_input_size(self):
        return self.cell_descriptors.shape[1]

### Load Dataset a Initialize NN model

In [59]:
current_data = data

# Split dataset -> 20% testing, 80% training
# Stratified split = each dataset has equal amounts of each class (saved in column 'target')
train, test = train_test_split(current_data, test_size = 0.2, stratify = current_data['target']) 

# Initlize dataset for NN
config.activation_fn = "sigmoid"
train_data = CellDataset(train, normalize_dataset())
test_data = CellDataset(test, normalize_dataset())

# Create NN and training class
mlp = MultiLayerPerceptron(train_data.get_input_size())
trainer = Trainer(config, mlp)

# Load dataset
trainer.load_dataset(train_data, test_data)

Using cpu device for training
751
751
6


### Wandb Project

In [4]:
wandb.login()

notebook_name = "NN-z1"

%env "WANDB_NOTEBOOK_NAME" notebook_name

wandb.init(
    project= notebook_name
)

env: "WANDB_NOTEBOOK_NAME"=notebook_name


In [ ]:
#%%wandb

no_epochs = 1000

for epoch in range(no_epochs):
    # Train model
    trainer.train()
    # Get metrics
    metrics = trainer.evaluate()

    #  wandb.log({"accuracy": accuracy, "loss": loss})

    